# RATIOED LEARNING FOR IMAGE RECONSTRUCTION AND GENERATION

# loads all required dependencies to build and load a model

In [1]:
import numpy as np
import time
from keras.models import Sequential,load_model
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D,GlobalAveragePooling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop,SGD
import matplotlib.pyplot as plt
from keras.models import load_model


Using TensorFlow backend.


CREATES A DEEP CONVOLUTIONAL GAN CLASS WITH METHODS CONSISTING OF BOTH GENERATOR AND ADVERSARIAL BLOCKS. 
THE ADVERSARIAL BLOCK HAS A DISCRIMINATOR AT THE TOP LAYER AND GENERATOR AT THE BOTTOM LAYER
THE GENERATOR UPSAMPLES A [1X100] GAUSSIAN OR UNIFORM SCALAR NOISE INTO [64X64] 2D ARRAY.
THE DISCRIMINATOR OUTPUT THE PROBABILITY OF A SAMPLE GENERATED BY GENERATOR AS FAKE OR REAL.

In [2]:
class DCGAN(object):
    def __init__(self, img_rows=64, img_cols=64, channel=1):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
        #self.D=load_model('discriminator.h5')
        
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In:64  x 64 x 1, depth = 1
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        print ('Discriminator Summary')
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
        #self.G =load_model('generator.h5')
        
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        
        self.G.add(Conv2DTranspose(int(depth/16), 5, padding='valid'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        
        self.G.add(Conv2DTranspose(int(depth/32), 5, padding='valid'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        print ('Generator summary')
        self.G.summary()
        return self.G

    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.00002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.00001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        print ('Adversarial Summary')
        self.AM.summary()
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM


CREATES A FACE_GAN CLASS WITH THE OBJECTS OF DEEP CONVOLUTION GAN CLASS. THE FACE_GAN CLASS HAS
METHODS SUCH AS FORWARD AND PLOT IMAGES. 

In [6]:
class FACE_GAN(object):
    def __init__(self):
        self.img_rows = 64
        self.img_cols = 64
        self.channel = 1
        self.a_ratio=2
        self.x_train=np.load('faces.npy')
        self.x_train = self.x_train.reshape(-1, self.img_rows,self.img_cols, 1)
        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

    def forward(self,start_step, train_steps, batch_size=256, save_interval=0,save_drive=False):
        noise_input = None
        for i in range(start_step,train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)
            print ("%d: [D loss: %f]" % (i, d_loss[0]))
            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            if i%self.a_ratio==0:
                a_loss = self.adversarial.train_on_batch(noise, y)
                print ("%d: [A loss: %f]" % (i, a_loss[0]))
            if i%500==0 and save_drive:
                self.generator.save('generator_{}.h5'.format(i))
                self.discriminator.save('discriminator_{}.h5'.format(i))
                FACE_GAN.plot_images(fake=True,save2file=True,step=i)
              


    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = "reconstructed_{}.png".format(step)
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "reconstructed_{}.png".format(step)
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()


LOADS THE FACE_GAN CLASS TO AN OBJECT 

In [7]:
face_reconstructor=FACE_GAN()

Discriminator Summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 64)        1664      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 256)        

TRAINS AND VISUALIZES THE GENERATOR & DISCRIMINATOR FOR FACES

In [8]:
face_reconstructor.forward(start_step=0,train_steps=5,save_drive=False)
face_reconstructor.plot_images(fake=True)

0: [D loss: 0.692908]
0: [A loss: 0.674885]
1: [D loss: 0.690379]
2: [D loss: 0.684367]


KeyboardInterrupt: 